In [ ]:
from tensorflow import keras # 텐션플로우로 부터 케라스를 입포트 합니다
keras.__version__# 텐션플로우 버전 확인

In [ ]:
#import tensorflow as tf
#tf.compat.v1.disable_v2_behavior()

In [ ]:
from google.colab import drive # 코렙의 드라이브 임포트 수행
import os, shutil #코렙으로 부터 os로 파일 읽거나, 폴더를 생성할때, 사용되는 os library 불러옴

drive.mount('/content/drive/') # 해당 경로를 드라이브로 마운트 수행
os.chdir('/content/drive/My Drive/PART1_ai_project_20210824_0830') #수행
!pwd# 확인

In [ ]:
import numpy as np # linear algebra
import pandas as pd # recall pd

import os #os 호출
for dirname, _, filenames in os.walk('/content/drive/My Drive/PART1_ai_project_20210824_0830/02_Projects/02_MinSeungJun'): #dir 와 파일들 불러들임
    for filename in filenames:
        os.path.join(dirname, filename)

In [ ]:
traindir='/content/drive/My Drive/PART1_ai_project_20210824_0830/01_Images/Train' #train data
validdir='/content/drive/My Drive/PART1_ai_project_20210824_0830/01_Images/Validation' # validation data
testdir='/content/drive/My Drive/PART1_ai_project_20210824_0830/01_Images/Test' #test data

In [ ]:
path='/content/drive/My Drive/PART1_ai_project_20210824_0830/01_Images/All'# 경로 설정
import cv2 # open cv?
import random #랜덤호출
import matplotlib.pyplot as plt #그림 
import matplotlib.image as mpimg #단일플롯에 다수 이미지 그리는거

plt.figure(figsize=(20,20)) #보여주기
for i in range(5): #5번
    file=random.choice(os.listdir(path)) # 랜덤으로 파일을 불러들인다?
    img_path=os.path.join(path,file) # 사진 경로가져오기
    image=mpimg.imread(img_path) #그리기
    ax=plt.subplot(1,5,i+1) # 부분 사진
    plt.imshow(image)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator #강사님이 말씀하진 이미지 제너레이터

train_data=ImageDataGenerator(rescale=1.0/255, zoom_range=0.2, shear_range=0.2) # train data 크기 조정
train_generator = train_data.flow_from_directory(directory=traindir,target_size=(64,64),class_mode='binary',batch_size=32) #어떤식으로 작용하는지는 모르겠습니다
test_data=ImageDataGenerator(rescale=1.0/255) #test data 크기 조정
valid_generator = test_data.flow_from_directory(directory=validdir,target_size=(64,64),class_mode='binary',batch_size=32) #어떤식으로 작용하는지는 모르겠습니다
test_generator = test_data.flow_from_directory(directory=testdir,target_size=(64,64),class_mode='binary',batch_size=32,shuffle=False) #어떤식으로 작용하는지는 모르겠습니다

print(valid_generator.class_indices)

In [ ]:
from tensorflow.keras import layers, Sequential
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
 # vgg 19 모델 사용이나, 새로운 모델로 적용할 꺼 같기에 코멘트 달지 않겠습니다

vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(64,64,3))

for layer in vgg19.layers:
    layer.trainable = False
    
vmodel = Sequential()
vmodel.add(vgg19)
vmodel.add(layers.Flatten())
vmodel.add(layers.Dense(1,activation='sigmoid'))
vmodel.summary()

vmodel.compile(optimizer='adam',loss='binary_crossentropy', metrics='accuracy')

In [ ]:
#기존
#history = vmodel.fit(train_generator,steps_per_epoch=len(train_generator)//8,epochs=20,validation_data=valid_generator)
#신규
history = vmodel.fit(train_generator,steps_per_epoch=len(train_generator)//8,epochs=10,validation_data=valid_generator, batch_size=128)

In [ ]:
def plot_history(history): # 그림그리기
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s] #loss 랑 val값들이랑 분류작업 밑에도 똑같은 작용일듯
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0: # loss값이 없다면
        print('Loss is missing in history') # 잘못됬다는걸 보여주는거겠죠?
        return 
    
    epochs = range(1,len(history.history[loss_list[0]]) + 1) # epoch 횟수..
    
    ## Loss
    plt.figure(1) #도화지
    for l in loss_list: #순서대로 표기
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
plot_history(history)